[![](https://img.shields.io/badge/Source%20on%20GitHub-orange)](https://github.com/laminlabs/lamin-spatial/blob/main/docs/vitessce.ipynb)

# Vitessce

This tutorial has been adopted from the data preparation examples in [the Vitessce documention](https://vitessce.github.io/vitessce-python).

## Setup

Install dependencies:

```python
pip install vitessce
pip install 'lamindb[jupyter,aws,bionty]'
```

In [ ]:
!lamin load laminlabs/lamindata  # load your instance

In [ ]:
from urllib.request import urlretrieve
from pathlib import Path
from anndata import read_h5ad
from vitessce import (
    VitessceConfig,
    Component as cm,
    AnnDataWrapper,
)
from vitessce.data_utils import (
    to_uint8,
    sort_var_axis,
    optimize_adata,
)
import lamindb as ln

In [ ]:
# [optional] track the current notebook or script
ln.settings.transform.stem_uid = "BZhZQ6uIbkWv"
ln.settings.transform.version = "1"
ln.track()

## Download and pre-process the dataset

For this example, we use a dataset from the COVID-19 Cell Atlas https://www.covid19cellatlas.org/index.healthy.html#habib17.

In [ ]:
# From https://github.com/vitessce/vitessce-python/blob/main/demos/habib-2017/src/convert_to_zarr.py
def convert_h5ad_to_zarr(input_path, output_path):
    adata = read_h5ad(input_path)
    adata = adata[:, adata.var["highly_variable"]].copy()
    leaf_list = sort_var_axis(adata.X, adata.var.index.values)
    adata = adata[:, leaf_list].copy()
    adata.layers["X_uint8"] = to_uint8(adata.X, norm_along="var")
    adata = optimize_adata(
        adata, obs_cols=["CellType"], obsm_keys=["X_umap"], layer_keys=["X_uint8"]
    )
    adata.write_zarr(output_path)

In [ ]:
adata_filepath = "./habib17.processed.h5ad"
if not Path(adata_filepath).exists():
    urlretrieve(
        "https://covid19.cog.sanger.ac.uk/habib17.processed.h5ad", adata_filepath
    )
zarr_filepath = "./hhabib_2017_nature_methods.anndata.zarr"

convert_h5ad_to_zarr(adata_filepath, zarr_filepath)

## Save dataset in LaminDB

In [ ]:
zarr_artifact = ln.Artifact(
    zarr_filepath,
    description="Habib et al., 2017 Nature Methods, zarr export optimized for Vitessce",
)
zarr_artifact.save()

## Create a VitessceConfig artifact

:::{note}

Here is [a note](https://lamin.ai/laminlabs/lamindata/transform/WDjio16cQsdW5zKv) on folder upload speed and we why chose to not use the `.export(to="s3")` functionality of Vitessce.

:::

Set up the configuration by adding the views and datasets of interest.

In [ ]:
vc = VitessceConfig(
    schema_version="1.0.15",
    name="Habib et al., 2017 Nature Methods",
    description=(
        "Archived frozen adult human post-mortem brain tissue profiled by snRNA-seq"
        " (DroNc-seq)"
    ),
)
dataset = vc.add_dataset(name="Habib 2017").add_object(
    AnnDataWrapper(
        adata_url=zarr_artifact.path.to_url(),
        obs_feature_matrix_path="layers/X_uint8",
        obs_embedding_paths=["obsm/X_umap"],
        obs_embedding_names=["UMAP"],
        obs_set_paths=["obs/CellType"],
        obs_set_names=["Cell Type"],
    )
)
obs_sets = vc.add_view(cm.OBS_SETS, dataset=dataset)
obs_sets_sizes = vc.add_view(cm.OBS_SET_SIZES, dataset=dataset)
scatterplot = vc.add_view(cm.SCATTERPLOT, dataset=dataset, mapping="UMAP")
heatmap = vc.add_view(cm.HEATMAP, dataset=dataset)
genes = vc.add_view(cm.FEATURE_LIST, dataset=dataset)
vc.layout(((scatterplot | obs_sets) / heatmap) | (obs_sets_sizes / genes))

# inspect the config
vc.to_dict()

## Save VitessceConfig to LaminDB

In [ ]:
from lamindb.integrations import save_vitessce_config

In [ ]:
vc_artifact = save_vitessce_config(vc, description="View Habib17 in Vitessce")

In [ ]:
vc_artifact.view_lineage()

In [ ]:
# [optional] save run report to share notebook with collaborators
# ln.finish()

In [ ]:
# clean up artifacts in CI run
zarr_artifact.delete(permanent=True)
vc_artifact.delete(permanent=True)